<a href="https://colab.research.google.com/github/nithinrk11/Feynn-Labs-Machine-Learning-Internship/blob/main/Project%203/ReciHealth_Obesity_Based_Recipe_Suggestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* The implementation of machine learning on the obesity dataset to predict obesity risk can be found at: [obesity_risk_prediction.ipynb](https://github.com/nithinrk11/Feynn-Labs-Machine-Learning-Internship/blob/main/Project%203/Data%20Prep%20and%20Model%20Training/obesity_risk_prediction.ipynb)
* Implementation of random forest classifer on the refined obesity risk dataset can be found at: [obesity_prediction_refined.ipynb](https://github.com/nithinrk11/Feynn-Labs-Machine-Learning-Internship/blob/main/Project%203/Data%20Prep%20and%20Model%20Training/obesity_prediction_refined.ipynb)
* Implementation of the pre-trained obesity prediction model can be found at: [obesity_pred_pkl.ipynb](https://github.com/nithinrk11/Feynn-Labs-Machine-Learning-Internship/blob/main/Project%203/Data%20Prep%20and%20Model%20Training/obesity_pred_pkl.ipynb)
* Data preprocessing of the recipe dataset can be found at: [Recipe_Obesity_dataset_prep.ipynb](https://github.com/nithinrk11/Feynn-Labs-Machine-Learning-Internship/blob/main/Project%203/Data%20Prep%20and%20Model%20Training/Recipe_Obesity_dataset_prep.ipynb)
* Machine learning on the refined recipe dataset can be found at: [Recipe_obese_recep_pred.ipynb](https://github.com/nithinrk11/Feynn-Labs-Machine-Learning-Internship/blob/main/Project%203/Data%20Prep%20and%20Model%20Training/Recipe_obese_recep_pred.ipynb)

> This notebook goes through the loading of the pre-trained obesity and recipe prediction model and predicting on random input data to classify recipes based on predicted obesity.

##Installing Dependencies

In [ ]:
import numpy as np
import pandas as pd
import pickle

In [ ]:
!git clone https://github.com/nithinrk11/Feynn-Labs-Machine-Learning-Internship.git
%cd Feynn-Labs-Machine-Learning-Internship

Cloning into 'Feynn-Labs-Machine-Learning-Internship'...
remote: Enumerating objects: 433, done.
remote: Counting objects: 100% (306/306), done.
remote: Compressing objects: 100% (213/213), done.
remote: Total 433 (delta 177), reused 164 (delta 89), pack-reused 127 (from 1)
Receiving objects: 100% (433/433), 16.11 MiB | 16.48 MiB/s, done.
Resolving deltas: 100% (217/217), done.
/content/Feynn-Labs-Machine-Learning-Internship/Feynn-Labs-Machine-Learning-Internship


##Load the trained models and the recipe dataset

In [ ]:
# Load the Random Forest model
with open('/content/Feynn-Labs-Machine-Learning-Internship/Project 3/trained_models/rf_model.pkl', 'rb') as f:
    rf_model = pickle.load(f)

# Load the label encoder
with open('/content/Feynn-Labs-Machine-Learning-Internship/Project 3/trained_models/label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

# Load the feature scaler
with open('/content/Feynn-Labs-Machine-Learning-Internship/Project 3/trained_models/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
# Load the trained recipe prediction model from the pickle file
with open('/content/Feynn-Labs-Machine-Learning-Internship/Project 3/trained_models/trained_obese_recep_model.pkl', 'rb') as file:
    loaded_models = pickle.load(file)
# Load the dataset
data = pd.read_csv('/content/Feynn-Labs-Machine-Learning-Internship/Project 3/filtered_datasets/obese_recipe_df.csv')

##Performing the predictions based on random input data on the Obesity prediction model

In [ ]:
# Define the new data point
new_data = {
    'Gender': [1],  # Male (0 for Female, 1 for Male)
    'Age': [15],
    'Height': [4.75],
    'Weight': [370],
    'family_history_with_overweight': [1],  # Yes (1) or No (0)
    'FCVC': [4],  # Food Consumption per Day
    'NCP': [3],  # Number of Main Meals
    'CAEC': [1],  # Always (1) or Sometimes (0) for 'Consumption of Food between Meals'
    'CH2O': [2],  # Daily Water Intake
    'FAF': [2],  # Physical Activity Frequency
    'TUE': [2],  # Time using technology
    'CALC': [0]   # Alcohol Consumption (No)
}

# Convert the new data into a DataFrame
new_data_df = pd.DataFrame(new_data)

# Scale the new data using the previously saved scaler
new_data_scaled = scaler.transform(new_data_df)

# Predict the obesity type using the loaded Random Forest model
prediction = rf_model.predict(new_data_scaled)



In [ ]:
# Decode the prediction (if necessary)
decoded_prediction = label_encoder.inverse_transform(prediction)

print(f"Predicted Obesity Type: {decoded_prediction[0]}")

Predicted Obesity Type: Obesity_Type_III


##Performing the predictions based on predicted output i.e., obesity type as the input data for the recipe prediction model.

In [ ]:
# Bug handling at the occurance of "Normal Weight"
if decoded_prediction[0] == 'Normal_Weight':
  decoded_prediction[0] = decoded_prediction[0].replace("_", " ")


# Example user input (current obesity type)
user_obesity_type = decoded_prediction[0]

# Prepare recommendations based on user obesity type
def recommend_recipes(user_obesity_type, data, models):
    recommendations = []

    for index, row in data.iterrows():
        new_recipe = pd.DataFrame({
            'Calories': [row['Calories']],
            'Sugar (g)': [row['Sugar (g)']],
            'Total Fat (g)': [row['Total Fat (g)']],
            'Saturated Fat (g)': [row['Saturated Fat (g)']]
        })

        # Predict for the user’s obesity type
        prediction = models[user_obesity_type].predict(new_recipe)

        if prediction[0] == 1:
            recommendations.append({
                'Recipe Name': row['Recipe Name'],
                'Cuisine': row['Cuisine'],  # Include cuisine information
                'Calories': row['Calories'],
                'Sugar (g)': row['Sugar (g)'],
                'Total Fat (g)': row['Total Fat (g)'],
                'Saturated Fat (g)': row['Saturated Fat (g)']
            })

    # Create DataFrame from recommendations
    recommendations_df = pd.DataFrame(recommendations)
    return recommendations_df

# Get recommendations for the user using the loaded models
recommended_recipes_df = recommend_recipes(user_obesity_type, data, loaded_models)

# Display the DataFrame with recommended recipes, their nutritional details, and cuisine
print(f"Recommended recipes for {user_obesity_type}:")
recommended_recipes_df


Recommended recipes for Obesity_Type_III:


,Recipe Name,Cuisine,Calories,Sugar (g),Total Fat (g),Saturated Fat (g)
0,Andhra Gunpowder,Andhra Pradesh,134.521833,4.477519,5.776929,3.226080
1,Ginger Chutney,Andhra Pradesh,75.018000,7.984223,0.785400,0.163840
2,Hyderabadi Lamb Biryani,Hyderabadi,19.324959,0.071394,0.705807,0.197278
3,"Prawns Curry, Kerala Style",Kerala,369.329583,0.393990,5.687433,1.419641
4,Kerala Style Spicy Tangy Fish Curry,Kerala,13.593500,0.227485,0.457745,0.100375
5,Oats Kanji - Oats Soup With Green Gram South I...,Kerala,517.770500,7.269615,3.842165,0.813590
6,punjabi mooli paratha recipe,Punjabi,31.974750,3.758943,0.173239,0.021455
7,rajasthani besan bhindi masala recipe,Rajasthani,195.084187,6.994077,2.776008,0.319766
8,Bengali Five Spice Blend,Bengali,54.076000,0.192050,3.218470,0.344560
9,Garam Masala,Bengali,70.441000,0.457730,1.785060,0.460445


Based on the suggested nutritional details, many of the recommended recipes seem to be suitable for individuals with Obesity Type III, particularly due to their low sugar and fat content. However, calorie content varies, so the total daily intake should be monitored to ensure it aligns with the individual's dietary needs.

Personalization: It's always best to consult a healthcare provider or a dietitian to tailor dietary recommendations to individual health needs and goals. They can provide more personalized advice based on a complete assessment of health status and dietary requirements.